<a href="https://colab.research.google.com/github/hbprosper/AIMS/blob/main/notebooks/01_pinn_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PhotonOrbitSolver Training
> African Institute of Mathematical Sciences (AIMS), Cape Town, South Africa<br>
> Created: March 2025 Claire David, Tlotlo M. Oepeng, Harrison B. Prosper

## Introduction
This notebook trains a Physics-Informed Neural Network (PINN) [1, 2] to solve the following nonlinear ordinary differential equation (ODE):

\begin{align}
  \overset{\textstyle\cdot\cdot}{u}  \: + \: u - \: 3 \: \frac{u^2}{2}  & = \: 0,
\end{align}

This equation describes the orbit of photons in a Schwarzschild spacetime about a spherically symmetric body of mass $M$.

### Notation
Our variable of interest here is

\begin{align}
  u & = \: \frac{r_s }{ r},
\end{align}

where $r_s$ is the Schwarzschild radius is defined as $r_s = \: \frac{2 G M}{c^2},$ where $G$ is Newton's gravitational constant and $c$ is the speed of light in vacuum.


If $C$ is the proper circumference of a circle centered at the center of mass,
in a Schwarzschild spacetime, the radial coordinate $r \equiv \frac{C}{2\pi}$ differs from the proper radial distance.


The overdot here ($\overset{\textstyle\cdot\cdot}{u}$) indicates differentiation with respect to $\phi$, the azimuthal angle in a spherical polar coordinate system, $(r, \theta, \phi)$. Here $\theta$ is set to $\pi \, / \, 2$ without loss of generality.


The initial conditions are
\begin{align}
u\,(0) &= u_0 \\
\overset{\textstyle\cdot}{u}\,(0) &= v_0.
\end{align}


### Approach
The ODE is solved using a PINN following the approach in [3]. The neural network is described by the function $g_\beta(\phi; u_0, v_0),$ where $\beta$ are the network's trainable weights.  
  
We use the following Ansatz from the theory of connections (ToC) [4] that incorporates the initial conditions explicitly:

\begin{align}
    u(\phi; u_0, v_0)  &= u_0 + g_\beta(\phi; u_0, v_0) - g_\beta(0; u_0, v_0) + \phi \left[ v_0 - \dot{g}_\beta(0; u_0, v_0) \right], \\[1ex]
    \dot{u}(\phi; u_0, v_0) &= v_0 + \dot{g}_\beta(\phi; u_0, v_0) - \dot{g}_\beta(0; u_0, v_0),
\end{align}

### References
[1] B. Moseley, [Deep Learning in Scientific Computing (2023)](https://camlab.ethz.ch/teaching/deep-learning-in-scientific-computing-2023.html), ETH Zürich, Computational and Applied Mathematics Laboratory (CAMLab)  
[2] S. Cuomo *et al*., *Scientific Machine Learning through Physics-Informed Neural Networks: Where we are and What's next*, [arXiv:2201.05624](https://doi.org/10.48550/arXiv.2201.05624)  
[3] Aditi S. Krishnapriyan, Amir Gholami, Shandian Zhe, Robert M. Kirby, Michael W. Mahoney, *Characterizing possible failure modes in physics-informed neural networks*, NIPS'21: Proceedings of the 35th International Conference on Neural Information Processing Systems; [arXiv:2109.01050](https://arxiv.org/abs/2109.01050)  
[4] D. Mortari, *The Theory of Connections: Connecting Points*, Mathematics, vol. 5, no. 57, 2017.


## Local installation `pinn4bhoc`
  ```bash
      git clone https://github.com/soot-bit/pinn4bhoc
      cd pinn4bhoc
      pip install -e .
  ```
## Google Colab installation `pinn4bhoc`
  1. Assign Colab working folder to string `COLAB_FOLDER` in notebook.
  2. Execute cell below.

In [10]:
!ls pinn4bhoc/

clone2colab.ipynb  notebooks  pinn4bhoc  pyproject.toml  README.md  runs


In [1]:
# uninstall pinn4bhoc if installed
%rm -rf pinn4bhoc
%pip uninstall -y pinn4bhoc

# clone pinn4bhoc
!git clone https://github.com/soot-bit/pinn4bhoc
%cd pinn4bhoc

# install pinn4 bhoc
%pip install -e .

Cloning into 'pinn4bhoc'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 222 (delta 129), reused 125 (delta 56), pack-reused 0 (from 0)
Receiving objects: 100% (222/222), 255.58 KiB | 3.41 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/pinn4bhoc
Obtaining file:///content/pinn4bhoc
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pinn4bhoc (pyproject.toml) ... done
  Created wheel for pinn4bhoc: filename=pinn4bhoc-0.1.0-0.editable-py3-none-any.whl size=4709 sha256=2fa2d2e2ef5b7bf083baedc567b7ea3005a8d055527f0732c7f5cb726737047e
  Stored in directory: /tmp/pip-ephem-wheel-cache-uqqhnznb/wheels/11/29/79/44bf54bc12b1609c3d73e9f19a907400fe89b63f221f5f0691
Successfully built pinn4bhoc


# Imports

In [2]:
import os
import sys
import importlib
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR

# PINN library
import pinn4bhoc.nn as mlp
import pinn4bhoc.utils.data as dat

## Setup

In [3]:
# -----------------------------
# Hardware
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}\n")

# -----------------------------
# Dataset Configuration
# -----------------------------
"""
For tests:
n_steps                    = 5        # Number of steps with constant LR
n_iterations_per_step      = 2000     # Training iterations per LR step
monitor_every_n_iterations = 200      # Frequency of logging/monitoring
"""

name = 'fcnn_sobol'

# choose whether to create or load a configuration file
load_existing_config = False

if load_existing_config:
    config = mlp.Config(f'{name}.yaml')
else:
    # create new configuration
    config = mlp.Config(name)

    # Phi segement size
    config('dPhi', 0.1)

    # Bounds
    #                       Phi     u0    v0
    config('lower_bounds', [0.0,  0.10, -1.0])
    config('upper_bounds', [config('dPhi'), 0.99,  1.0])

    # training configuration
    # -----------------------------------------
    config('dataset_size_exponent', 16)
    config('train_size', 2**config('dataset_size_exponent'))
    config('val_size',     5000)  # validation sample size
    config('batch_size',   2048)  #
    config('monitor_step', 2000)  # monitor training every n (=10) iterations
    config('delete', True)        # delete losses file before training, if True

    # optimizer / scheduler configuration
    # -----------------------------------------
    # a step comprises a given number of iterations
    config('num_steps', 5)        # number of training steps
    config('num_iters_per_step', 200_000)
    config('base_lr', 3e-3)       # initial learning rate
    config('gamma', 0.24)         # learning rate scale factor

    config('val_cost_drop_threshold', 0.005)
    print(f'\nSave configuration to file {config.cfg_filename}\n')

    config.save()

# Total number of iterations
config('num_iterations', config('num_iters_per_step') * config('num_steps'))

print(config)

# -----------------------------
# Live Plotting Options
# -----------------------------
live_display  = False    # to plot cost evolution during training

Using device: cuda


Save configuration to file runs/2025-11-07_2044/fcnn_sobol_config.yaml

name: fcnn_sobol
file:
  losses: runs/2025-11-07_2044/fcnn_sobol_losses.csv
  params: runs/2025-11-07_2044/fcnn_sobol_params.pth
  init_params: runs/2025-11-07_2044/fcnn_sobol_init_params.pth
  plots: runs/2025-11-07_2044/fcnn_sobol_plots.png
dPhi: 0.1
lower_bounds:
- 0.0
- 0.1
- -1.0
upper_bounds:
- 0.1
- 0.99
- 1.0
dataset_size_exponent: 16
train_size: 65536
val_size: 5000
batch_size: 2048
monitor_step: 2000
delete: true
num_steps: 5
num_iters_per_step: 200000
base_lr: 0.003
gamma: 0.24
val_cost_drop_threshold: 0.005
num_iterations: 1000000



# Datasets

In [4]:
# -----------------------------
# Point Generation
# -----------------------------
# Training data
sampling_strategy = 'sobol'

if sampling_strategy.lower() == "sobol":
    data_train = dat.SobolSample(config('lower_bounds'),
                                  config('upper_bounds'),
                                  num_points_exp=config('dataset_size_exponent')
    )
elif sampling_strategy.lower() == "uniform":
    data_train = dat.UniformSample(config('lower_bounds'),
                                   config('upper_bounds'),
                                   num_points=2**config('dataset_size_exponent')
    )
else:
    raise ValueError("sampling_strategy must be one of: sobol, uniform")
print()

# Validation data
data_val = dat.UniformSample(config('lower_bounds'),
                             config('upper_bounds'),
                             num_points=config('val_size')
)

# -----------------------------
# PINN Datasets
# -----------------------------
# Dataset for training: full tensorized subset from the raw sampling
print('\n===> Creating: train_dataset')
train_dataset = dat.Dataset(data_train,
                            start=0, end=config('train_size'),
                            verbose=1,
                            device=device
)

# Training subset of same size as validation dataset
print('\n===> Creating: train_valsize_dataset')
train_valsize_dataset = dat.Dataset(data_train,
                                    start=0, end=config('train_size'),
                                    random_sample_size=config('val_size'),
                                    verbose=1,
                                    device=device
)

# Dataset for validation: tensorized points from the raw uniform sampling
print('\n===> Creating: val_dataset')
val_dataset = dat.Dataset(data_val,
                          start=0, end=config('val_size'),
                          verbose=1,
                          device=device
)

  SobolSample
  65536 Sobol points created.

  UniformSample
  5000 uniformly sampled points created.

===> Creating: train_dataset
  Type               : Dataset
  Shape of phi_vals  : torch.Size([65536, 1])
  Shape of init_conds: torch.Size([65536, 2])

===> Creating: train_valsize_dataset
  Type               : Dataset
  Shape of phi_vals  : torch.Size([5000, 1])
  Shape of init_conds: torch.Size([5000, 2])

===> Creating: val_dataset
  Type               : Dataset
  Shape of phi_vals  : torch.Size([5000, 1])
  Shape of init_conds: torch.Size([5000, 2])


# DataLoaders

In [5]:
# Loader for main training batches
print('\n===> Creating: train_loader')
train_loader = dat.DataLoader(train_dataset,
                              batch_size=config('batch_size'),
                              num_iterations=config('num_iterations'),
                              shuffle=True
)

# Loader for evaluating validation cost (single batch of val_size)
print('\n===> Creating: val_loader')
val_loader = dat.DataLoader(val_dataset,
                            batch_size=config('val_size')
)

# Loader for evaluating training cost with val-sized batch
print('\n===> Creating: train_valsize_loader')
train_valsize_loader = dat.DataLoader(train_valsize_dataset,
                                      batch_size=config('val_size')
)


===> Creating: train_loader
DataLoader
  Number of iterations has been specified
  maxiter:         1000000
  batch_size:         2048
  shuffle_step:         32


===> Creating: val_loader
DataLoader
  maxiter:               1
  batch_size:         5000
  shuffle_step:          1


===> Creating: train_valsize_loader
DataLoader
  maxiter:               1
  batch_size:         5000
  shuffle_step:          1



## Model, Solution, Objective

In [6]:
# Model Instantiation
print('\n===> Creating model...\n')

fcnn_model = mlp.FCNN().to(device)
pinn_soln  = mlp.Solution(fcnn_model).to(device)
pinn_obj   = mlp.Objective(pinn_soln).to(device)

print(fcnn_model)
print(f'Number of parameters: {mlp.count_trainable_parameters(fcnn_model)}')

# Save model with initial weights
init_model_filename = config('file/init_params')
fcnn_model.save(init_model_filename)

print(f"\n===> Saved initial model weights to:\n\t{init_model_filename}")


===> Creating model...

FCNN(
  (model): ModuleList(
    (0): Linear(in_features=3, out_features=75, bias=True)
    (1): Sin()
    (2): Linear(in_features=75, out_features=75, bias=True)
    (3): Sin()
    (4): Linear(in_features=75, out_features=75, bias=True)
    (5): Sin()
  )
  (output_layer): Linear(in_features=75, out_features=1, bias=True)
)
Number of parameters: 11776

===> Saved initial model weights to:
	runs/2025-11-07_2044/fcnn_sobol_init_params.pth


# Scheduler
Using a multistep scheduler parametrized with $\gamma$ (initially 0.24).


In [7]:
# Instantiate optimizer with base learning rate
print("\n===> Creating optimizer...\n")
print(f"    Base learning rate: {config('base_lr'):10.1e}")
optimizer = torch.optim.Adam(pinn_soln.parameters(), lr=config('base_lr'))

# Learning rate milestones (after n_step iterations)
n_milestones = config('num_steps') - 1
print(f'    Number of milestones: {n_milestones:5d}\n')
milestones = [n * config('num_iters_per_step') for n in range(config('num_steps'))]

print("\n===> Creating scheduler...\n")
# Drop first entry of milestones list because it contains the base LR
scheduler = MultiStepLR(optimizer, milestones=milestones[1:], gamma=config('gamma'))

mlp.print_milestones_and_lrs(config('base_lr'),
                             config('num_steps'),
                             milestones,
                             config('gamma'),
                             n_max_iterations=config('num_iterations'))


===> Creating optimizer...

    Base learning rate:    3.0e-03
    Number of milestones:     4


===> Creating scheduler...

Step | Milestone | LR
-----------------------------
   0 |         0 | 3.0e-03   
-----------------------------
   1 |    200000 | 7.2e-04   
   2 |    400000 | 1.7e-04   
   3 |    600000 | 4.1e-05   
   4 |    800000 | 1.0e-05   

Total number of iterations:    1000000



# Training Loop

In [ ]:
mlp.train_pinn(train_loader, val_loader, train_valsize_loader,
               optimizer, scheduler, pinn_obj,
               display_costs=live_display,
               model_filename=config('file/params'),
               log_filename=config('file/losses'),
               plot_filename=config('file/plots'),
               monitor_every_n_iterations=config('monitor_step'),
               drop_threshold=config('val_cost_drop_threshold')
)

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:179.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[Iteration        0]  LR:  3.0e-03  |  Iter/s:   1.0  |  Time: 00:00:01
   └── Cost [Train / Val / Best Val]:  3.148e-02  /  3.137e-02  /  3.137e-02
[Iteration     2000]  LR:  3.0e-03  |  Iter/s: 124.1  |  Time: 00:00:16
   └── Cost [Train / Val / Best Val]:  1.446e-05  /  1.426e-05  /  1.426e-05
[Iteration     4000]  LR:  3.0e-03  |  Iter/s: 128.5  |  Time: 00:00:31
   └── Cost [Train / Val / Best Val]:  8.343e-06  /  8.390e-06  /  8.390e-06
[Iteration     6000]  LR:  3.0e-03  |  Iter/s: 129.9  |  Time: 00:00:46
   └── Cost [Train / Val / Best Val]:  9.575e-06  /  1.003e-05  /  8.390e-06
[Iteration     8000]  LR:  3.0e-03  |  Iter/s: 130.5  |  Time: 00:01:01
   └── Cost [Train / Val / Best Val]:  2.498e-06  /  2.734e-06  /  2.734e-06
[Iteration    10000]  LR:  3.0e-03  |  Iter/s: 131.0  |  Time: 00:01:16
   └── Cost [Train / Val / Best Val]:  7.007e-06  /  6.932e-06  /  2.734e-06
[Iteration    12000]  LR:  3.0e-03  |  Iter/s: 131.2  |  Time: 00:01:31
   └── Cost [Train / Val / Best Va